###I. Model

1. Resnet


In [ ]:
import torch.nn as nn
import torch

In [ ]:
class nonlinearly (nn.Module):
  def __init__(self):
    super().__init__()

    def forward(self,x):
      return x * torch.sidmoid(x)

NameError: name 'nn' is not defined

In [ ]:
class ResConv1DBlock(nn.Module):
  def __init__(self, n_in, n_state, dilation = 1, activation = 'silu', norm=None, dropout = None):
    super().__init__()
    padding = dilation
    self.norm = norm
    if norm == "LN":
        self.norm1 = nn.LayerNorm(n_in)
        self.norm2 = nn.LayerNorm(n_in)
    elif norm == "GN":
        self.norm1 = nn.GroupNorm(num_groups=32, num_channels=n_in, eps=1e-6, affine=True)
        self.norm2 = nn.GroupNorm(num_groups=32, num_channels=n_in, eps=1e-6, affine=True)
    elif norm == "BN":
        self.norm1 = nn.BatchNorm1d(num_features=n_in, eps=1e-6, affine=True)
        self.norm2 = nn.BatchNorm1d(num_features=n_in, eps=1e-6, affine=True)

    else:
        self.norm1 = nn.Identity()
        self.norm2 = nn.Identity()
    if activation == "relu":
        self.activation1 = nn.ReLU()
        self.activation2 = nn.ReLU()

    elif activation == "silu":
        self.activation1 = nonlinearly()
        self.activation2 = nonlinearly()

    elif activation == "gelu":
        self.activation1 = nn.GELU()
        self.activation2 = nn.GELU()

    self.conv1 = nn.Conv1d(n_in, n_state, kernel_size = 3, stride = 1, padding = 1, dilation = 1)
    self.conv2 = nn.Conv1d(n_state, n_in, kernel_size=1, stride =1, padding =0)

  def forward(self, x):
    x_orig = x
    if self.norm == "LN":
        x = self.norm1(x.transpose(-2, -1))
        x = self.activation1(x.transpose(-2, -1))
    else:
        x = self.norm1(x)
        x = self.activation1(x)

    x = self.conv1(x)

    if self.norm == "LN":
        x = self.norm2(x.transpose(-2, -1))
        x = self.activation2(x.transpose(-2, -1))
    else:
        x = self.norm2(x)
        x = self.activation2(x)

    x = self.conv2(x)
    x = x + x_orig
    return x


NameError: name 'nn' is not defined

In [ ]:
class Resnet1D(nn.Module):
    def __init__(self, n_in, n_depth, dilation_growth_rate=1, reverse_dilation=True, activation='relu', norm=None):
        super().__init__()

        blocks = [ResConv1DBlock(n_in, n_in, dilation=dilation_growth_rate ** depth, activation=activation, norm=norm) for depth in range(n_depth)]
        if reverse_dilation:
            blocks = blocks[::-1]

        self.model = nn.Sequential(*blocks)

    def forward(self, x):
        return self.model(x)

In [ ]:
m = Resnet1D(16, 33,dilation_growth_rate=15)
input = torch.randn(20, 16, 50)
output = m(input)
output.shape

2. Encoder and decoder

In [ ]:
import torch.nn as nn
class Encoder(nn.Module):
  def __init__(self,
               input_emb_width = 3,
               output_emb_width = 512,
               down_t = 3,
               stride_t =2,
               width = 512,
               depth = 3,
               dilation_growth_rate = 3,
               activation = 'relu',
               norm = 'None'):
      super().__init__()
      blocks = []
      filter_t, pad_t = stride_t * 2, stride_t //2
      blocks.append(nn.Conv1d(input_emb_width, width, kernel_size = 3 , stride =1 , padding =1))
      blocks.append(nn.ReLU())

      for i in range(down_t):
            input_dim = width
            block = nn.Sequential(
                nn.Conv1d(input_dim, width, filter_t, stride_t, pad_t),
                Resnet1D(width, depth, dilation_growth_rate, activation=activation, norm=norm),
            )
            blocks.append(block)
      blocks.append(nn.Conv1d(width, output_emb_width, 3, 1, 1))
      self.model = nn.Sequential(*blocks)



  def forward(self, x):
          return self.model(x)

#test shape
encode = Encoder(norm = "GN")
x = torch.rand(50,3,20)
x = encode(x)
x.shape


In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 input_emb_width = 3,
                 output_emb_width = 512,
                 down_t = 3,
                 stride_t = 2,
                 width = 512,
                 depth = 3,
                 dilation_growth_rate = 3,
                 activation='relu',
                 norm=None):
        super().__init__()
        blocks = []
        filter_t, pad_t = stride_t * 2, stride_t // 2
        blocks.append(nn.Conv1d(output_emb_width, width, 3, 1, 1))
        blocks.append(nn.ReLU())
        for i in range(down_t):
            out_dim = width
            block = nn.Sequential(
                Resnet1D(width, depth, dilation_growth_rate, reverse_dilation=True, activation=activation, norm=norm),
                nn.Upsample(scale_factor=2, mode='nearest'),
                nn.Conv1d(width, out_dim, 3, 1, 1)
            )
            blocks.append(block)
        blocks.append(nn.Conv1d(width, width, 3, 1, 1))
        blocks.append(nn.ReLU())
        blocks.append(nn.Conv1d(width, input_emb_width, 3, 1, 1))
        self.model = nn.Sequential(*blocks)

    def forward(self, x):
        return self.model(x)
decode = Decoder(norm = "GN")
x = torch.rand(50,3,12)
x = encode(x)
x.shape

3. QuantizeCNN


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class QuantizeEMAReset(nn.Module):
  def __init__(self, nb_code, code_dim,args):
    super().__init__()
    self.nb_code = nb_code
    self.code_dim = code_dim
    self.mu = args.mu
    self.reset_codebook()

  def reset_codebook(self):
    self.init = False
    self.code_sum = None
    self.code_count = None
    self.register_buffer('codebook', torch.zeros(self.nb_code, self.code_dim).cuda()) #register a tensor

  def _tile(self, x):
        nb_code_x, code_dim = x.shape
        if nb_code_x < self.nb_code:
            n_repeats = (self.nb_code + nb_code_x - 1) // nb_code_x
            std = 0.01 / np.sqrt(code_dim)
            out = x.repeat(n_repeats, 1)
            out = out + torch.randn_like(out) * std
        else :
            out = x
        return out
  def init_codebook(self,x):
    out = self._tile(x)
    self.codebook = out[:self.nb_code]
    self.code_sum = self.codebook.clone()
    self.code_count = torch.ones(self.nb_code,device=self.codebook.device)
  @torch.no_grad()
  def compute_perplexity(self, code_idx) :
      # Calculate new centres
    code_onehot = torch.zeros(self.nb_code, code_idx.shape[0], device=code_idx.device)  # nb_code, N * L
    code_onehot.scatter_(0, code_idx.view(1, code_idx.shape[0]), 1)

    code_count = code_onehot.sum(dim=-1)  # nb_code
    prob = code_count / torch.sum(code_count)
    perplexity = torch.exp(-torch.sum(prob * torch.log(prob + 1e-7)))
    return perplexity

  @torch.no_grad()
  def update_codebook(self, x, code_idx):

    code_onehot = torch.zeros(self.nb_code, x.shape[0], device=x.device)  # nb_code, N * L
    code_onehot.scatter_(0, code_idx.view(1, x.shape[0]), 1)

    code_sum = torch.matmul(code_onehot, x)  # nb_code, w
    code_count = code_onehot.sum(dim=-1)  # nb_code

    out = self._tile(x)
    code_rand = out[:self.nb_code]

        # Update centres
    self.code_sum = self.mu * self.code_sum + (1. - self.mu) * code_sum  # w, nb_code
    self.code_count = self.mu * self.code_count + (1. - self.mu) * code_count  # nb_code

    usage = (self.code_count.view(self.nb_code, 1) >= 1.0).float()
    code_update = self.code_sum.view(self.nb_code, self.code_dim) / self.code_count.view(self.nb_code, 1)

    self.codebook = usage * code_update + (1 - usage) * code_rand
    prob = code_count / torch.sum(code_count)
    perplexity = torch.exp(-torch.sum(prob * torch.log(prob + 1e-7)))


    return perplexity
  def preprocess(self,x):
    x = x.permute(0,2,1).contiguous()
    x = x.view(-1, x.shape[-1])
    return x

  def quantize(self, x):
    # Calculate latent code x_l
    k_w = self.codebook.t()

    distance = torch.sum(x ** 2, dim=-1, keepdim=True)
    - 2 * torch.matmul(x, k_w) + torch.sum(k_w ** 2, dim=0, keepdim=True)  # (N * L, b)

    _, code_idx = torch.min(distance, dim=-1)
    return code_idx

  def dequantize(self,code_idx):
    x = F.embedding(code_idx, self.codebook)
    return x

  def forward(self, x):
    N, width, T = x.shape

    #preprocess
    x = self.preprocess(x)

    if self.trainning and not self.init:
      self.init_codebook(x)

    code_idx = self.quantize(x)
    x_d = self.dequantize(code_idx)

    if self.training:
      perplexity = self.update_codebook(x, code_idx)
    else:
      perplexity = self.compute_perplexity(code_idx)

     # Loss
    commit_loss = F.mse_loss(x, x_d.detach())

    # Passthrough
    x_d = x + (x_d - x).detach()

    # Postprocess
    x_d = x_d.view(N, T, -1).permute(0, 2, 1).contiguous()   #(N, DIM, T)

    return x_d, commit_loss, perplexity


NameError: name 'nn' is not defined

4. Vq-vae

In [ ]:
class VQVAE_251(nn.Module):
  def __init__(self,
               args,
               nb_code=1024,
               code_dim = 512,
               output_emb_width=512,
               down_t = 3,
               stride_t=2,
               width = 512,
               depth = 3,
               dilation_growth_rate = 3,
               activation = 'relu',
               norm = "GN"):
    super().__init__()
    self.code_dim = code_dim
    self.num_code = nb_code
    self.quant = args.quantizer
    self.encoder = Encoder(251 if args.dataname == 'kit' else 263, output_emb_width, down_t, stride_t, width, depth, dilation_growth_rate, activation=activation, norm=norm)
    self.decoder = Decoder(251 if args.dataname == 'kit' else 263, output_emb_width, down_t, stride_t, width, depth, dilation_growth_rate, activation=activation, norm=norm)
    self.quantizer = QuantizeEMAReset(nb_code, code_dim, args)

    def preprocess(self, x):
        # (bs, T, Jx3) -> (bs, Jx3, T)
        x = x.permute(0,2,1).float()
        return x


    def postprocess(self, x):
        # (bs, Jx3, T) ->  (bs, T, Jx3)
        x = x.permute(0,2,1)
        return x

    def encode(self,x):
      N, T, _= x.shape
      x_in = self.preprocess(x)
      x_encoder = self.encoder(x_in)
      x_encoder = self.postprocess(x_encoder)
      x_encoder = x_encoder.contiguous().view(-1, x_encoder.shape[-1])  # (NT, C)
      code_idx = self.quantizer.quantize(x_encoder)
      code_idx = code_idx.view(N,-1)
      return code_idx

    def forward(self, x):
      x_in = self.preprocess(x)
      ##encode
      x_encoder = self.encoder(x_in)

      ## quantization
      x_quantized, loss, perplexity  = self.quantizer(x_encoder)

      ##decode
      x_decoder = self.decoder(x_quantized)
      x_out = self.postprocess(x_decoder)
      return x_out, loss, perplexity


    def forward_decoder(self, x):
      """
      Function is used to decode the code_idx when inference.
      """

      x_d = self.quantizer.dequantize(x)
      x_d = x_d.view(1, -1, self.code_dim).permute(0, 2, 1).contiguous()

      # decoder
      x_decoder = self.decoder(x_d)
      x_out = self.postprocess(x_decoder)
      return x_out


In [ ]:
class HumanVQVAE(nn.Module):
    def __init__(self,
                 args,
                 nb_code=512,
                 code_dim=512,
                 output_emb_width=512,
                 down_t=3,
                 stride_t=2,
                 width=512,
                 depth=3,
                 dilation_growth_rate=3,
                 activation='relu',
                 norm=None):

        super().__init__()

        self.nb_joints = 21 if args.dataname == 'kit' else 22
        self.vqvae = VQVAE_251(args, nb_code, code_dim, output_emb_width, down_t, stride_t, width, depth, dilation_growth_rate, activation=activation, norm=norm)

    def encode(self, x):
        b, t, c = x.size()
        quants = self.vqvae.encode(x) # (N, T)
        return quants

    def forward(self, x):

        x_out, loss, perplexity = self.vqvae(x)

        return x_out, loss, perplexity

    def forward_decoder(self, x):
        x_out = self.vqvae.forward_decoder(x)
        return x_out

5.pos_encoding


In [ ]:
"""
Various positional encodings for the transformer.
"""
import math
import torch
from torch import nn

def PE1d_sincos(seq_length, dim):
    """
    :param d_model: dimension of the model
    :param length: length of positions
    :return: length*d_model position matrix
    """
    if dim % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(dim))
    pe = torch.zeros(seq_length, dim)
    position = torch.arange(0, seq_length).unsqueeze(1)
    div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                         -(math.log(10000.0) / dim)))
    pe[:, 0::2] = torch.sin(position.float() * div_term)
    pe[:, 1::2] = torch.cos(position.float() * div_term)

    return pe.unsqueeze(1)


class PositionEmbedding(nn.Module):
    """
    Absolute pos embedding (standard), learned.
    """
    def __init__(self, seq_length, dim, dropout = 0.1, grad=False):
        super().__init__()
        self.embed = nn.Parameter(data=PE1d_sincos(seq_length, dim), requires_grad=grad)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # x.shape: bs, seq_len, feat_dim
        l = x.shape[1]
        x = x.permute(1, 0, 2) + self.embed[:l].expand(x.permute(1, 0, 2).shape)
        x = self.dropout(x.permute(1, 0, 2))
        return x



6.t2m_trans


In [ ]:
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.distributions import Categorical

In [ ]:
class CausalCrossConditionSelfAttention(nn.Module):
  def __init__(self,
               embed_dim =512,
               block_size = 16,
               n_head=8,
               drop_out_rate = 0.1):
    super().__init__()
    assert embed_dim % 8 == 0
    # k, q , v
    self.key = nn.Linear(embed_dim, embed_dim)
    self.query = nn.Linear(embed_dim, embed_dim)
    self.value = nn.Linear(embed_dim, embed_dim)

    self.attn_drop = nn.Dropout(drop_out_rate)
    self.resid_drop = nn.Dropout(drop_out_rate)

    self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size)).view(1, 1, block_size, block_size))
    self.n_head = n_head

    def forward(self,x):
      B, T, C = x.size()

       # calculate query, key, values for all heads in batch and move head forward to be the batch dim
      k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
      q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
      v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

      att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
      att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
      att = F.softmax(att, dim=-1)
      att = self.attn_drop(att)

      y = att @ v  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
      y = y.transpose(1, 2).contiguous().view(B, T, C) ## -> (B, T , C)

      # output projection
      y = self.resid_drop(self.proj(y))
      return y


In [ ]:
class Block(nn.Module):
  def __init__(self,
               embed_dim =512,
               block_size = 16,
               n_head = 8,
               drop_out_rate = 0.1,
               fc_rate = 4):
    super().__init__()
    self.LayerNorm1 = nn.LayerNorm(embed_dim)
    self.LayerNorm2 = nn.LayerNorm(embed_dim)
    self.attn = CausalCrossConditionSelfAttention(embed_dim, block_size, n_head, drop_out_rate)
    self.mlp = nn.Sequential(
        nn.Linear(embed_dim, fc_rate * embed_dim),
        nn.GELU(),
        nn.Linear(fc_rate* embed_dim, embed_dim),
        nn.Dropout(drop_out_rate),
    )

    def forward(self,x):
      x = x + self.attn(self.LayerNorm1(x))
      x = x + self.mlp(self.LayerNorm2(x))
      return x


In [ ]:
class CrossCondTransBase(nn.Module):
    def __init__(self,
                num_vq=1024,
                embed_dim=512,
                clip_dim=512,
                block_size=16,
                num_layers=2,
                n_head=8,
                drop_out_rate=0.1,
                fc_rate=4):
        super().__init__()
        self.tok_emb = nn.Embedding(num_vq + 2, embed_dim)
        self.cond_emb = nn.Linear(clip_dim, embed_dim)
        self.pos_embedding = nn.Embedding(block_size, embed_dim)
        self.drop = nn.Dropout(drop_out_rate)
        # transformer block
        self.blocks = nn.Sequential(*[Block(embed_dim, block_size, n_head, drop_out_rate, fc_rate) for _ in range(num_layers)])
        self.pos_embed = PositionEmbedding(block_size, embed_dim, 0.0, False)
        self.block_size = 16

        self.apply(self._init_weights)

        def get_block_size(self):
          return self.block_size

        def _init_weights(self, module):
          if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
              module.bias.data.zero_()
          elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

        def forward(self, idx, clip_feature):
          if len(idx) == 0:
            token_embeddings = self.cond_emb(clip_feature).unsqueeze(1)
          else:
            b, t = idx.size()
            assert t <= self.block_size, "Cannot forward, model block size is exhausted."
            # forward the Trans model
            token_embeddings = self.tok_emb(idx)
            token_embeddings = torch.cat([self.cond_emb(clip_feature).unsqueeze(1), token_embeddings], dim=1)

          x = self.pos_embed(token_embeddings)
          x = self.blocks(x)

          return x

In [ ]:
class CrossCondTransHead(nn.Module):
    def __init__(self,
                num_vq=1024,
                embed_dim=512,
                block_size=16,
                num_layers=2,
                n_head=8,
                drop_out_rate=0.1,
                fc_rate=4):
        super().__init__()

        self.blocks = nn.Sequential(*[Block(embed_dim, block_size, n_head, drop_out_rate, fc_rate) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_vq + 1, bias=False)
        self.block_size = block_size

        self.apply(self._init_weights)

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, x):
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits

In [ ]:
class Text2Motion_Transformer(nn.Module):
  def __init__(self,
               num_vq = 1024,
               embed_dim = 512,
               clip_dim=512,
               block_size = 16,
               num_layers = 2,
               n_head = 8,
               drop_out_rate = 0.1,
               fc_rate = 4):
    super().__init__()
    self.trans_base = CrossCondTransBase(num_vq, embed_dim, clip_dim, block_size, num_layers, n_head, drop_out_rate, fc_rate)
    self.trans_head = CrossCondTransHead(num_vq, embed_dim, block_size, num_layers, n_head, drop_out_rate, fc_rate)
    self.block_size = block_size
    self.num_vq = num_vq

    def get_block_size(self):
      return self.block_size

    def forward(self, idxs, clip_feature):
      feat = self.trans_base(idxs, clip_feature)
      logits = self.trans_head(feat)
      return logits

    def sample(self, clip_feature, if_categorical = True):
      for k in range(self.blocksize):
        if k==0:
          x=[]
        else:
          x = xs
          logits = self.forward(x, clip_feature)
          logits = logits[:,-1,:]
          probs = F.softmax(logits, dim=-1)
          if if_categorical:
                dist = Categorical(probs)
                idx = dist.sample()
                if idx == self.num_vq:
                    break
                idx = idx.unsqueeze(-1)
          else:
                _, idx = torch.topk(probs, k=1, dim=-1)
                if idx[0] == self.num_vq:
                    break
          if k == 0:
                xs = idx
          else:
                xs = torch.cat((xs, idx), dim=1)

          if k == self.block_size - 1:
                return xs[:, :-1]
        return xs



In [ ]:
# @title modules

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence

def init_weight(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear) or isinstance(m, nn.ConvTranspose1d):
        nn.init.xavier_normal_(m.weight)
        # m.bias.data.fill_(0.01)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)


class MovementConvEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MovementConvEncoder, self).__init__()
        self.main = nn.Sequential(
            nn.Conv1d(input_size, hidden_size, 4, 2, 1),
            nn.Dropout(0.2, inplace=True),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv1d(hidden_size, output_size, 4, 2, 1),
            nn.Dropout(0.2, inplace=True),
            nn.LeakyReLU(0.2, inplace=True),
        )
        self.out_net = nn.Linear(output_size, output_size)
        self.main.apply(init_weight)
        self.out_net.apply(init_weight)

    def forward(self, inputs):
        inputs = inputs.permute(0, 2, 1)
        outputs = self.main(inputs).permute(0, 2, 1)
        # print(outputs.shape)
        return self.out_net(outputs)



class TextEncoderBiGRUCo(nn.Module):
    def __init__(self, word_size, pos_size, hidden_size, output_size, device):
        super(TextEncoderBiGRUCo, self).__init__()
        self.device = device

        self.pos_emb = nn.Linear(pos_size, word_size)
        self.input_emb = nn.Linear(word_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.output_net = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_size, output_size)
        )

        self.input_emb.apply(init_weight)
        self.pos_emb.apply(init_weight)
        self.output_net.apply(init_weight)
        self.hidden_size = hidden_size
        self.hidden = nn.Parameter(torch.randn((2, 1, self.hidden_size), requires_grad=True))

    # input(batch_size, seq_len, dim)
    def forward(self, word_embs, pos_onehot, cap_lens):
        num_samples = word_embs.shape[0]

        pos_embs = self.pos_emb(pos_onehot)
        inputs = word_embs + pos_embs
        input_embs = self.input_emb(inputs)
        hidden = self.hidden.repeat(1, num_samples, 1)

        cap_lens = cap_lens.data.tolist()
        emb = pack_padded_sequence(input_embs, cap_lens, batch_first=True)

        gru_seq, gru_last = self.gru(emb, hidden)

        gru_last = torch.cat([gru_last[0], gru_last[1]], dim=-1)

        return self.output_net(gru_last)


class MotionEncoderBiGRUCo(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device):
        super(MotionEncoderBiGRUCo, self).__init__()
        self.device = device

        self.input_emb = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.output_net = nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(hidden_size, output_size)
        )

        self.input_emb.apply(init_weight)
        self.output_net.apply(init_weight)
        self.hidden_size = hidden_size
        self.hidden = nn.Parameter(torch.randn((2, 1, self.hidden_size), requires_grad=True))

    # input(batch_size, seq_len, dim)
    def forward(self, inputs, m_lens):
        num_samples = inputs.shape[0]

        input_embs = self.input_emb(inputs)
        hidden = self.hidden.repeat(1, num_samples, 1)

        cap_lens = m_lens.data.tolist()
        emb = pack_padded_sequence(input_embs, cap_lens, batch_first=True, enforce_sorted=False)

        gru_seq, gru_last = self.gru(emb, hidden)

        gru_last = torch.cat([gru_last[0], gru_last[1]], dim=-1)

        return self.output_net(gru_last)

In [ ]:
# @title evaluator_wrapper.py

import torch
from os.path import join as pjoin
import numpy as np
from models.modules import MovementConvEncoder, TextEncoderBiGRUCo, MotionEncoderBiGRUCo
from utils.word_vectorizer import POS_enumerator

def build_models(opt):
    movement_enc = MovementConvEncoder(opt.dim_pose-4, opt.dim_movement_enc_hidden, opt.dim_movement_latent)
    text_enc = TextEncoderBiGRUCo(word_size=opt.dim_word,
                                  pos_size=opt.dim_pos_ohot,
                                  hidden_size=opt.dim_text_hidden,
                                  output_size=opt.dim_coemb_hidden,
                                  device=opt.device)

    motion_enc = MotionEncoderBiGRUCo(input_size=opt.dim_movement_latent,
                                      hidden_size=opt.dim_motion_hidden,
                                      output_size=opt.dim_coemb_hidden,
                                      device=opt.device)

    checkpoint = torch.load(pjoin(opt.checkpoints_dir, opt.dataset_name, 'text_mot_match', 'model', 'finest.tar'),
                            map_location=opt.device)
    movement_enc.load_state_dict(checkpoint['movement_encoder'])
    text_enc.load_state_dict(checkpoint['text_encoder'])
    motion_enc.load_state_dict(checkpoint['motion_encoder'])
    print('Loading Evaluation Model Wrapper (Epoch %d) Completed!!' % (checkpoint['epoch']))
    return text_enc, motion_enc, movement_enc


class EvaluatorModelWrapper(object):

    def __init__(self, opt):

        if opt.dataset_name == 't2m':
            opt.dim_pose = 263
        elif opt.dataset_name == 'kit':
            opt.dim_pose = 251
        else:
            raise KeyError('Dataset not Recognized!!!')

        opt.dim_word = 300
        opt.max_motion_length = 196
        opt.dim_pos_ohot = len(POS_enumerator)
        opt.dim_motion_hidden = 1024
        opt.max_text_len = 20
        opt.dim_text_hidden = 512
        opt.dim_coemb_hidden = 512

        # print(opt)

        self.text_encoder, self.motion_encoder, self.movement_encoder = build_models(opt)
        self.opt = opt
        self.device = opt.device

        self.text_encoder.to(opt.device)
        self.motion_encoder.to(opt.device)
        self.movement_encoder.to(opt.device)

        self.text_encoder.eval()
        self.motion_encoder.eval()
        self.movement_encoder.eval()

    # Please note that the results does not following the order of inputs
    def get_co_embeddings(self, word_embs, pos_ohot, cap_lens, motions, m_lens):
        with torch.no_grad():
            word_embs = word_embs.detach().to(self.device).float()
            pos_ohot = pos_ohot.detach().to(self.device).float()
            motions = motions.detach().to(self.device).float()

            '''Movement Encoding'''
            movements = self.movement_encoder(motions[..., :-4]).detach()
            m_lens = m_lens // self.opt.unit_length
            motion_embedding = self.motion_encoder(movements, m_lens)

            '''Text Encoding'''
            text_embedding = self.text_encoder(word_embs, pos_ohot, cap_lens)
        return text_embedding, motion_embedding

    # Please note that the results does not following the order of inputs
    def get_motion_embeddings(self, motions, m_lens):
        with torch.no_grad():
            motions = motions.detach().to(self.device).float()

            align_idx = np.argsort(m_lens.data.tolist())[::-1].copy()
            motions = motions[align_idx]
            m_lens = m_lens[align_idx]

            '''Movement Encoding'''
            movements = self.movement_encoder(motions[..., :-4]).detach()
            m_lens = m_lens // self.opt.unit_length
            motion_embedding = self.motion_encoder(movements, m_lens)
        return motion_embedding

In [ ]:
# @title smpl
!pip install smplx
# This code is based on https://github.com/Mathux/ACTOR.git
import numpy as np
import torch

import contextlib

from smplx import SMPLLayer as _SMPLLayer
from smplx.lbs import vertices2joints


# action2motion_joints = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 21, 24, 38]
# change 0 and 8
action2motion_joints = [8, 1, 2, 3, 4, 5, 6, 7, 0, 9, 10, 11, 12, 13, 14, 21, 24, 38]

from utils.config import SMPL_MODEL_PATH, JOINT_REGRESSOR_TRAIN_EXTRA

JOINTSTYPE_ROOT = {"a2m": 0, # action2motion
                   "smpl": 0,
                   "a2mpl": 0, # set(smpl, a2m)
                   "vibe": 8}  # 0 is the 8 position: OP MidHip below

JOINT_MAP = {
    'OP Nose': 24, 'OP Neck': 12, 'OP RShoulder': 17,
    'OP RElbow': 19, 'OP RWrist': 21, 'OP LShoulder': 16,
    'OP LElbow': 18, 'OP LWrist': 20, 'OP MidHip': 0,
    'OP RHip': 2, 'OP RKnee': 5, 'OP RAnkle': 8,
    'OP LHip': 1, 'OP LKnee': 4, 'OP LAnkle': 7,
    'OP REye': 25, 'OP LEye': 26, 'OP REar': 27,
    'OP LEar': 28, 'OP LBigToe': 29, 'OP LSmallToe': 30,
    'OP LHeel': 31, 'OP RBigToe': 32, 'OP RSmallToe': 33, 'OP RHeel': 34,
    'Right Ankle': 8, 'Right Knee': 5, 'Right Hip': 45,
    'Left Hip': 46, 'Left Knee': 4, 'Left Ankle': 7,
    'Right Wrist': 21, 'Right Elbow': 19, 'Right Shoulder': 17,
    'Left Shoulder': 16, 'Left Elbow': 18, 'Left Wrist': 20,
    'Neck (LSP)': 47, 'Top of Head (LSP)': 48,
    'Pelvis (MPII)': 49, 'Thorax (MPII)': 50,
    'Spine (H36M)': 51, 'Jaw (H36M)': 52,
    'Head (H36M)': 53, 'Nose': 24, 'Left Eye': 26,
    'Right Eye': 25, 'Left Ear': 28, 'Right Ear': 27
}

JOINT_NAMES = [
    'OP Nose', 'OP Neck', 'OP RShoulder',
    'OP RElbow', 'OP RWrist', 'OP LShoulder',
    'OP LElbow', 'OP LWrist', 'OP MidHip',
    'OP RHip', 'OP RKnee', 'OP RAnkle',
    'OP LHip', 'OP LKnee', 'OP LAnkle',
    'OP REye', 'OP LEye', 'OP REar',
    'OP LEar', 'OP LBigToe', 'OP LSmallToe',
    'OP LHeel', 'OP RBigToe', 'OP RSmallToe', 'OP RHeel',
    'Right Ankle', 'Right Knee', 'Right Hip',
    'Left Hip', 'Left Knee', 'Left Ankle',
    'Right Wrist', 'Right Elbow', 'Right Shoulder',
    'Left Shoulder', 'Left Elbow', 'Left Wrist',
    'Neck (LSP)', 'Top of Head (LSP)',
    'Pelvis (MPII)', 'Thorax (MPII)',
    'Spine (H36M)', 'Jaw (H36M)',
    'Head (H36M)', 'Nose', 'Left Eye',
    'Right Eye', 'Left Ear', 'Right Ear'
]


# adapted from VIBE/SPIN to output smpl_joints, vibe joints and action2motion joints
class SMPL(_SMPLLayer):
    """ Extension of the official SMPL implementation to support more joints """

    def __init__(self, model_path=SMPL_MODEL_PATH, **kwargs):
        kwargs["model_path"] = model_path

        # remove the verbosity for the 10-shapes beta parameters
        with contextlib.redirect_stdout(None):
            super(SMPL, self).__init__(**kwargs)

        J_regressor_extra = np.load(JOINT_REGRESSOR_TRAIN_EXTRA)
        self.register_buffer('J_regressor_extra', torch.tensor(J_regressor_extra, dtype=torch.float32))
        vibe_indexes = np.array([JOINT_MAP[i] for i in JOINT_NAMES])
        a2m_indexes = vibe_indexes[action2motion_joints]
        smpl_indexes = np.arange(24)
        a2mpl_indexes = np.unique(np.r_[smpl_indexes, a2m_indexes])

        self.maps = {"vibe": vibe_indexes,
                     "a2m": a2m_indexes,
                     "smpl": smpl_indexes,
                     "a2mpl": a2mpl_indexes}

    def forward(self, *args, **kwargs):
        smpl_output = super(SMPL, self).forward(*args, **kwargs)

        extra_joints = vertices2joints(self.J_regressor_extra, smpl_output.vertices)
        all_joints = torch.cat([smpl_output.joints, extra_joints], dim=1)

        output = {"vertices": smpl_output.vertices}

        for joinstype, indexes in self.maps.items():
            output[joinstype] = all_joints[:, indexes]

        return output

In [ ]:
# @title rotation2xyz
# This code is based on https://github.com/Mathux/ACTOR.git
import torch
import utils.rotation_conversions as geometry


from models.smpl import SMPL, JOINTSTYPE_ROOT
# from .get_model import JOINTSTYPES
JOINTSTYPES = ["a2m", "a2mpl", "smpl", "vibe", "vertices"]


class Rotation2xyz:
    def __init__(self, device, dataset='amass'):
        self.device = device
        self.dataset = dataset
        self.smpl_model = SMPL().eval().to(device)

    def __call__(self, x, mask, pose_rep, translation, glob,
                 jointstype, vertstrans, betas=None, beta=0,
                 glob_rot=None, get_rotations_back=False, **kwargs):
        if pose_rep == "xyz":
            return x

        if mask is None:
            mask = torch.ones((x.shape[0], x.shape[-1]), dtype=bool, device=x.device)

        if not glob and glob_rot is None:
            raise TypeError("You must specify global rotation if glob is False")

        if jointstype not in JOINTSTYPES:
            raise NotImplementedError("This jointstype is not implemented.")

        if translation:
            x_translations = x[:, -1, :3]
            x_rotations = x[:, :-1]
        else:
            x_rotations = x

        x_rotations = x_rotations.permute(0, 3, 1, 2)
        nsamples, time, njoints, feats = x_rotations.shape

        # Compute rotations (convert only masked sequences output)
        if pose_rep == "rotvec":
            rotations = geometry.axis_angle_to_matrix(x_rotations[mask])
        elif pose_rep == "rotmat":
            rotations = x_rotations[mask].view(-1, njoints, 3, 3)
        elif pose_rep == "rotquat":
            rotations = geometry.quaternion_to_matrix(x_rotations[mask])
        elif pose_rep == "rot6d":
            rotations = geometry.rotation_6d_to_matrix(x_rotations[mask])
        else:
            raise NotImplementedError("No geometry for this one.")

        if not glob:
            global_orient = torch.tensor(glob_rot, device=x.device)
            global_orient = geometry.axis_angle_to_matrix(global_orient).view(1, 1, 3, 3)
            global_orient = global_orient.repeat(len(rotations), 1, 1, 1)
        else:
            global_orient = rotations[:, 0]
            rotations = rotations[:, 1:]

        if betas is None:
            betas = torch.zeros([rotations.shape[0], self.smpl_model.num_betas],
                                dtype=rotations.dtype, device=rotations.device)
            betas[:, 1] = beta
            # import ipdb; ipdb.set_trace()
        out = self.smpl_model(body_pose=rotations, global_orient=global_orient, betas=betas)

        # get the desirable joints
        joints = out[jointstype]

        x_xyz = torch.empty(nsamples, time, joints.shape[1], 3, device=x.device, dtype=x.dtype)
        x_xyz[~mask] = 0
        x_xyz[mask] = joints

        x_xyz = x_xyz.permute(0, 2, 3, 1).contiguous()

        # the first translation root at the origin on the prediction
        if jointstype != "vertices":
            rootindex = JOINTSTYPE_ROOT[jointstype]
            x_xyz = x_xyz - x_xyz[:, [rootindex], :, :]

        if translation and vertstrans:
            # the first translation root at the origin
            x_translations = x_translations - x_translations[:, :, [0]]

            # add the translation to all the joints
            x_xyz = x_xyz + x_translations[:, None, :, :]

        if get_rotations_back:
            return x_xyz, rotations, global_orient
        else:
            return x_xyz


# II dataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/EricGuo5513/HumanML3D.git

Cloning into 'HumanML3D'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 204 (delta 50), reused 36 (delta 33), pack-reused 135
Receiving objects: 100% (204/204), 132.86 MiB | 15.67 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
# @title download dataset
!tar -xvjf /content/drive/MyDrive/Motion-dataset/ACCAD.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/TotalCapture.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/HDM05.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/TCDHands.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/SFU.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/BMLmovi.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/CMU.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/MoSh.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/EKUT.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/KIT.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/EyesJapanDataset.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/BMLhandball.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/Transitions.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/PosePrior.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/HumanEva.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/SSM.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/DFaust.tar.bz2
!tar -xvjf /content/drive/MyDrive/Motion-dataset/BMLrub.tar.bz2

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
KIT/10/WalkInClockwiseCircle09_poses.npz
KIT/10/RightTurn01_poses.npz
KIT/10/WalkInClockwiseCircle02_poses.npz
KIT/10/LeftTurn04_poses.npz
KIT/10/WalkInClockwiseCircle08_poses.npz
KIT/10/WalkInCounterClockwiseCircle03_poses.npz
KIT/10/WalkInCounterClockwiseCircle04_poses.npz
KIT/10/WalkingStraightBackwards06_poses.npz
KIT/10/WalkInClockwiseCircle01_poses.npz
KIT/10/LeftTurn01_poses.npz
KIT/10/WalkingStraightBackwards05_poses.npz
KIT/10/LeftTurn09_poses.npz
KIT/10/WalkInClockwiseCircle07_poses.npz
KIT/10/WalkingStraightBackwards08_poses.npz
KIT/10/RightTurn08_poses.npz
KIT/10/RightTurn05_poses.npz
KIT/10/WalkingStraightBackwards02_poses.npz
KIT/10/WalkInClockwiseCircle05_poses.npz
KIT/10/WalkingStraightBackwards03_poses.npz
LICENSE.txt
Eyes_Japan_Dataset/
Eyes_Japan_Dataset/frederic/
Eyes_Japan_Dataset/frederic/walk-04-fast-frederic_poses.npz
Eyes_Japan_Dataset/frederic/walk-06-catwalk-frederic_poses.npz
Eyes_Japan_Dataset/kaiwa/


In [ ]:
!tar -xf /content/drive/MyDrive/Motion-dataset/model/dmpls.tar.xz

In [ ]:
!tar -xf /content/drive/MyDrive/Motion-dataset/model/smplh.tar.xz

^C


In [ ]:
!mkdir amass_data

In [ ]:
import shutil

def move_directory(source_dir, target_dir):
    # Move the source directory to the target directory
    shutil.move(source_dir, target_dir)

# Example usage
source_directory = '/content/ACCAD'
target_directory = '/content/amass_data'
move_directory(source_directory, target_directory)
source_directory = '/content/BMLhandball'
move_directory(source_directory, target_directory)
source_directory = '/content/BMLmovi'
move_directory(source_directory, target_directory)
source_directory = '/content/BioMotionLab_NTroje'
move_directory(source_directory, target_directory)
source_directory = '/content/CMU'
move_directory(source_directory, target_directory)
source_directory = '/content/DFaust_67'
move_directory(source_directory, target_directory)
source_directory = '/content/EKUT'
move_directory(source_directory, target_directory)
source_directory = '/content/Eyes_Japan_Dataset'
move_directory(source_directory, target_directory)
source_directory = '/content/HumanEva'
move_directory(source_directory, target_directory)
source_directory = '/content/MPI_HDM05'
move_directory(source_directory, target_directory)
source_directory = '/content/MPI_Limits'
move_directory(source_directory, target_directory)
source_directory = '/content/SSM_synced'
move_directory(source_directory, target_directory)
source_directory = '/content/TCD_handMocap'
move_directory(source_directory, target_directory)
source_directory = '/content/Transitions_mocap'
move_directory(source_directory, target_directory)
source_directory = '/content/MPI_mosh'
move_directory(source_directory, target_directory)
source_directory = '/content/SFU'
move_directory(source_directory, target_directory)
source_directory = '/content/KIT'
move_directory(source_directory, target_directory)
source_directory = '/content/TotalCapture'
move_directory(source_directory, target_directory)

In [ ]:
!unzip /content/HumanML3D/pose_data/humanact12.zip

Archive:  /content/HumanML3D/pose_data/humanact12.zip
   creating: humanact12/
   creating: humanact12/humanact12/
  inflating: humanact12/humanact12/P01G01R01F0001T0064A0101.npy  
  inflating: humanact12/humanact12/P01G01R01F0069T0143A0102.npy  
  inflating: humanact12/humanact12/P01G01R01F0151T0190A0103.npy  
  inflating: humanact12/humanact12/P01G01R01F0449T0505A0201.npy  
  inflating: humanact12/humanact12/P01G01R01F0718T0825A0301.npy  
  inflating: humanact12/humanact12/P01G01R01F0839T0955A0401.npy  
  inflating: humanact12/humanact12/P01G01R01F1114T1267A0501.npy  
  inflating: humanact12/humanact12/P01G01R01F1316T1416A0502.npy  
  inflating: humanact12/humanact12/P01G01R01F1444T1577A0601.npy  
  inflating: humanact12/humanact12/P01G01R01F1580T1665A0602.npy  
  inflating: humanact12/humanact12/P01G01R01F1683T1740A0603.npy  
  inflating: humanact12/humanact12/P01G01R01F1777T1844A0604.npy  
  inflating: humanact12/humanact12/P01G01R02F0001T0071A0604.npy  
  inflating: humanact12/hum

In [ ]:
source_directory = '/content/humanact12'
move_directory(source_directory, target_directory)

In [ ]:
# @title raw_pose_processing
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin
import sys
sys.path.append("/content/HumanML3D/human_body_prior/tools")
from omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

# Choose the device to run the body model on.
comp_device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

sys.path.append("/content/HumanML3D/human_body_prior/body_model")
from body_model import BodyModel

male_bm_path = '/content/drive/MyDrive/Motion-dataset/smplh/male/model.npz'
male_dmpl_path = '/content/drive/MyDrive/Motion-dataset/dmpls/male/model.npz'

female_bm_path = '/content/drive/MyDrive/Motion-dataset/smplh/female/model.npz'
female_dmpl_path = '/content/drive/MyDrive/Motion-dataset/dmpls/female/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

male_bm = BodyModel(bm_fname = male_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=male_dmpl_path).to(comp_device)
faces = c2c(male_bm.f)

female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=female_dmpl_path).to(comp_device)

paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('./amass_data'):
#     print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[2]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))


save_root = '=./pose_data'
save_folders = [folder.replace('./amass_data', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0],
                            [0.0, 1.0, 0.0]])
ex_fps = 20
def amass_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    try:
        fps = bdata['mocap_framerate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps

    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    if bdata['gender'] == 'male':
        bm = male_bm
    else:
        bm = female_bm
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)

    bdata_poses = bdata['poses'][::down_sample,...]
    bdata_trans = bdata['trans'][::down_sample,...]
    body_parms = {
            'root_orient': torch.Tensor(bdata_poses[:, :3]).to(comp_device),
            'pose_body': torch.Tensor(bdata_poses[:, 3:66]).to(comp_device),
            'pose_hand': torch.Tensor(bdata_poses[:, 66:]).to(comp_device),
            'trans': torch.Tensor(bdata_trans).to(comp_device),
            'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=len(bdata_trans), axis=0)).to(comp_device),
        }

    with torch.no_grad():
        body = bm(**body_parms)
    pose_seq_np = body.Jtr.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)


    np.save(save_path, pose_seq_np_n)
    return fps
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

import time
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace('./amass_data', './pose_data')
        save_path = save_path[:-3] + 'npy'
        fps = amass_to_pose(path, save_path)

    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Processing: SSM_synced: 100%|██████████| 30/30 [00:06<00:00,  4.48it/s]


Processed / All (fps 59): 30/15246


Processing: MPI_HDM05: 100%|██████████| 215/215 [05:31<00:00,  1.54s/it]


Processed / All (fps 120): 245/15246


...

Processed / All (fps 120): 11014/15246


Processing: KIT: 100%|██████████| 4232/4232 [23:50<00:00,  2.96it/s]


Processed / All (fps 100): 15246/15246


  0%|          | 1/14616 [00:00<08:25, 28.93it/s]


FileNotFoundError: [Errno 2] No such file or directory: './pose_data/humanact12/humanact12/P11G01R02F1812T1847A0402.npy'

In [ ]:
def swap_left_right(data):
    assert len(data.shape) == 3 and data.shape[-1] == 3
    data = data.copy()
    data[..., 0] *= -1
    right_chain = [2, 5, 8, 11, 14, 17, 19, 21]
    left_chain = [1, 4, 7, 10, 13, 16, 18, 20]
    left_hand_chain = [22, 23, 24, 34, 35, 36, 25, 26, 27, 31, 32, 33, 28, 29, 30]
    right_hand_chain = [43, 44, 45, 46, 47, 48, 40, 41, 42, 37, 38, 39, 49, 50, 51]
    tmp = data[:, right_chain]
    data[:, right_chain] = data[:, left_chain]
    data[:, left_chain] = tmp
    if data.shape[1] > 24:
        tmp = data[:, right_hand_chain]
        data[:, right_hand_chain] = data[:, left_hand_chain]
        data[:, left_hand_chain] = tmp
    return data
index_path = '/content/HumanML3D/index.csv'
save_dir = '/content/joints'
index_file = pd.read_csv(index_path)
total_amount = index_file.shape[0]
fps = 20

for i in tqdm(range(total_amount)):
    source_path = index_file.loc[i]['source_path']
    new_name = index_file.loc[i]['new_name']
    data = np.load(source_path)
    start_frame = index_file.loc[i]['start_frame']
    end_frame = index_file.loc[i]['end_frame']
    if 'humanact12' not in source_path:
        if 'Eyes_Japan_Dataset' in source_path:
            data = data[3*fps:]
        if 'MPI_HDM05' in source_path:
            data = data[3*fps:]
        if 'TotalCapture' in source_path:
            data = data[1*fps:]
        if 'MPI_Limits' in source_path:
            data = data[1*fps:]
        if 'Transitions_mocap' in source_path:
            data = data[int(0.5*fps):]
        data = data[start_frame:end_frame]
        data[..., 0] *= -1

    data_m = swap_left_right(data)
#     save_path = pjoin(save_dir, )
    np.save(pjoin(save_dir, new_name), data)
    np.save(pjoin(save_dir, 'M'+new_name), data_m)

100%|██████████| 14616/14616 [01:35<00:00, 152.99it/s]


In [ ]:
source_directory = '/content/pose_data'
target_directory = '/content/drive/MyDrive/Motion-dataset'
move_directory(source_directory, target_directory)
source_directory = '/content/joints'
move_directory(source_directory, target_directory)

In [ ]:
# @title motiton_representation
from os.path import join as pjoin
import sys

from skeleton import Skeleton
import numpy as np
import os
from quaternion import *
sys.path.append('/content/HumanML3D')
from paramUtil import *
import torch
from tqdm import tqdm
import os

def uniform_skeleton(positions, target_offset):
    src_skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')
    src_offset = src_skel.get_offsets_joints(torch.from_numpy(positions[0]))
    src_offset = src_offset.numpy()
    tgt_offset = target_offset.numpy()
    # print(src_offset)
    # print(tgt_offset)
    '''Calculate Scale Ratio as the ratio of legs'''
    src_leg_len = np.abs(src_offset[l_idx1]).max() + np.abs(src_offset[l_idx2]).max()
    tgt_leg_len = np.abs(tgt_offset[l_idx1]).max() + np.abs(tgt_offset[l_idx2]).max()

    scale_rt = tgt_leg_len / src_leg_len
    # print(scale_rt)
    src_root_pos = positions[:, 0]
    tgt_root_pos = src_root_pos * scale_rt

    '''Inverse Kinematics'''
    quat_params = src_skel.inverse_kinematics_np(positions, face_joint_indx)
    # print(quat_params.shape)

    '''Forward Kinematics'''
    src_skel.set_offset(target_offset)
    new_joints = src_skel.forward_kinematics_np(quat_params, tgt_root_pos)
    return new_joints

def process_file(positions, feet_thre):
    # (seq_len, joints_num, 3)
    #     '''Down Sample'''
    #     positions = positions[::ds_num]

    '''Uniform Skeleton'''
    positions = uniform_skeleton(positions, tgt_offsets)

    '''Put on Floor'''
    floor_height = positions.min(axis=0).min(axis=0)[1]
    positions[:, :, 1] -= floor_height
    #     print(floor_height)

    #     plot_3d_motion("./positions_1.mp4", kinematic_chain, positions, 'title', fps=20)

    '''XZ at origin'''
    root_pos_init = positions[0]
    root_pose_init_xz = root_pos_init[0] * np.array([1, 0, 1])
    positions = positions - root_pose_init_xz

    # '''Move the first pose to origin '''
    # root_pos_init = positions[0]
    # positions = positions - root_pos_init[0]

    '''All initially face Z+'''
    r_hip, l_hip, sdr_r, sdr_l = face_joint_indx
    across1 = root_pos_init[r_hip] - root_pos_init[l_hip]
    across2 = root_pos_init[sdr_r] - root_pos_init[sdr_l]
    across = across1 + across2
    across = across / np.sqrt((across ** 2).sum(axis=-1))[..., np.newaxis]

    # forward (3,), rotate around y-axis
    forward_init = np.cross(np.array([[0, 1, 0]]), across, axis=-1)
    # forward (3,)
    forward_init = forward_init / np.sqrt((forward_init ** 2).sum(axis=-1))[..., np.newaxis]

    #     print(forward_init)

    target = np.array([[0, 0, 1]])
    root_quat_init = qbetween_np(forward_init, target)
    root_quat_init = np.ones(positions.shape[:-1] + (4,)) * root_quat_init

    positions_b = positions.copy()

    positions = qrot_np(root_quat_init, positions)

    #     plot_3d_motion("./positions_2.mp4", kinematic_chain, positions, 'title', fps=20)

    '''New ground truth positions'''
    global_positions = positions.copy()

    # plt.plot(positions_b[:, 0, 0], positions_b[:, 0, 2], marker='*')
    # plt.plot(positions[:, 0, 0], positions[:, 0, 2], marker='o', color='r')
    # plt.xlabel('x')
    # plt.ylabel('z')
    # plt.axis('equal')
    # plt.show()

    """ Get Foot Contacts """

    def foot_detect(positions, thres):
        velfactor, heightfactor = np.array([thres, thres]), np.array([3.0, 2.0])

        feet_l_x = (positions[1:, fid_l, 0] - positions[:-1, fid_l, 0]) ** 2
        feet_l_y = (positions[1:, fid_l, 1] - positions[:-1, fid_l, 1]) ** 2
        feet_l_z = (positions[1:, fid_l, 2] - positions[:-1, fid_l, 2]) ** 2
        #     feet_l_h = positions[:-1,fid_l,1]
        #     feet_l = (((feet_l_x + feet_l_y + feet_l_z) < velfactor) & (feet_l_h < heightfactor)).astype(np.float)
        feet_l = ((feet_l_x + feet_l_y + feet_l_z) < velfactor).astype(np.float32)

        feet_r_x = (positions[1:, fid_r, 0] - positions[:-1, fid_r, 0]) ** 2
        feet_r_y = (positions[1:, fid_r, 1] - positions[:-1, fid_r, 1]) ** 2
        feet_r_z = (positions[1:, fid_r, 2] - positions[:-1, fid_r, 2]) ** 2
        #     feet_r_h = positions[:-1,fid_r,1]
        #     feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor) & (feet_r_h < heightfactor)).astype(np.float)
        feet_r = (((feet_r_x + feet_r_y + feet_r_z) < velfactor)).astype(np.float32)
        return feet_l, feet_r
    #
    feet_l, feet_r = foot_detect(positions, feet_thre)
    # feet_l, feet_r = foot_detect(positions, 0.002)

    '''Quaternion and Cartesian representation'''
    r_rot = None

    def get_rifke(positions):
        '''Local pose'''
        positions[..., 0] -= positions[:, 0:1, 0]
        positions[..., 2] -= positions[:, 0:1, 2]
        '''All pose face Z+'''
        positions = qrot_np(np.repeat(r_rot[:, None], positions.shape[1], axis=1), positions)
        return positions

    def get_quaternion(positions):
        skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
        # (seq_len, joints_num, 4)
        quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=False)

        '''Fix Quaternion Discontinuity'''
        quat_params = qfix(quat_params)
        # (seq_len, 4)
        r_rot = quat_params[:, 0].copy()
        #     print(r_rot[0])
        '''Root Linear Velocity'''
        # (seq_len - 1, 3)
        velocity = (positions[1:, 0] - positions[:-1, 0]).copy()
        #     print(r_rot.shape, velocity.shape)
        velocity = qrot_np(r_rot[1:], velocity)
        '''Root Angular Velocity'''
        # (seq_len - 1, 4)
        r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
        quat_params[1:, 0] = r_velocity
        # (seq_len, joints_num, 4)
        return quat_params, r_velocity, velocity, r_rot

    def get_cont6d_params(positions):
        skel = Skeleton(n_raw_offsets, kinematic_chain, "cpu")
        # (seq_len, joints_num, 4)
        quat_params = skel.inverse_kinematics_np(positions, face_joint_indx, smooth_forward=True)

        '''Quaternion to continuous 6D'''
        cont_6d_params = quaternion_to_cont6d_np(quat_params)
        # (seq_len, 4)
        r_rot = quat_params[:, 0].copy()
        #     print(r_rot[0])
        '''Root Linear Velocity'''
        # (seq_len - 1, 3)
        velocity = (positions[1:, 0] - positions[:-1, 0]).copy()
        #     print(r_rot.shape, velocity.shape)
        velocity = qrot_np(r_rot[1:], velocity)
        '''Root Angular Velocity'''
        # (seq_len - 1, 4)
        r_velocity = qmul_np(r_rot[1:], qinv_np(r_rot[:-1]))
        # (seq_len, joints_num, 4)
        return cont_6d_params, r_velocity, velocity, r_rot

    cont_6d_params, r_velocity, velocity, r_rot = get_cont6d_params(positions)
    positions = get_rifke(positions)

    #     trejec = np.cumsum(np.concatenate([np.array([[0, 0, 0]]), velocity], axis=0), axis=0)
    #     r_rotations, r_pos = recover_ric_glo_np(r_velocity, velocity[:, [0, 2]])

    # plt.plot(positions_b[:, 0, 0], positions_b[:, 0, 2], marker='*')
    # plt.plot(ground_positions[:, 0, 0], ground_positions[:, 0, 2], marker='o', color='r')
    # plt.plot(trejec[:, 0], trejec[:, 2], marker='^', color='g')
    # plt.plot(r_pos[:, 0], r_pos[:, 2], marker='s', color='y')
    # plt.xlabel('x')
    # plt.ylabel('z')
    # plt.axis('equal')
    # plt.show()

    '''Root height'''
    root_y = positions[:, 0, 1:2]

    '''Root rotation and linear velocity'''
    # (seq_len-1, 1) rotation velocity along y-axis
    # (seq_len-1, 2) linear velovity on xz plane
    r_velocity = np.arcsin(r_velocity[:, 2:3])
    l_velocity = velocity[:, [0, 2]]
    #     print(r_velocity.shape, l_velocity.shape, root_y.shape)
    root_data = np.concatenate([r_velocity, l_velocity, root_y[:-1]], axis=-1)

    '''Get Joint Rotation Representation'''
    # (seq_len, (joints_num-1) *6) quaternion for skeleton joints
    rot_data = cont_6d_params[:, 1:].reshape(len(cont_6d_params), -1)

    '''Get Joint Rotation Invariant Position Represention'''
    # (seq_len, (joints_num-1)*3) local joint position
    ric_data = positions[:, 1:].reshape(len(positions), -1)

    '''Get Joint Velocity Representation'''
    # (seq_len-1, joints_num*3)
    local_vel = qrot_np(np.repeat(r_rot[:-1, None], global_positions.shape[1], axis=1),
                        global_positions[1:] - global_positions[:-1])
    local_vel = local_vel.reshape(len(local_vel), -1)

    data = root_data
    data = np.concatenate([data, ric_data[:-1]], axis=-1)
    data = np.concatenate([data, rot_data[:-1]], axis=-1)
    #     print(data.shape, local_vel.shape)
    data = np.concatenate([data, local_vel], axis=-1)
    data = np.concatenate([data, feet_l, feet_r], axis=-1)

    return data, global_positions, positions, l_velocity

# Recover global angle and positions for rotation data
# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def recover_root_rot_pos(data):
    rot_vel = data[..., 0]
    r_rot_ang = torch.zeros_like(rot_vel).to(data.device)
    '''Get Y-axis rotation from rotation velocity'''
    r_rot_ang[..., 1:] = rot_vel[..., :-1]
    r_rot_ang = torch.cumsum(r_rot_ang, dim=-1)

    r_rot_quat = torch.zeros(data.shape[:-1] + (4,)).to(data.device)
    r_rot_quat[..., 0] = torch.cos(r_rot_ang)
    r_rot_quat[..., 2] = torch.sin(r_rot_ang)

    r_pos = torch.zeros(data.shape[:-1] + (3,)).to(data.device)
    r_pos[..., 1:, [0, 2]] = data[..., :-1, 1:3]
    '''Add Y-axis rotation to root position'''
    r_pos = qrot(qinv(r_rot_quat), r_pos)

    r_pos = torch.cumsum(r_pos, dim=-2)

    r_pos[..., 1] = data[..., 3]
    return r_rot_quat, r_pos


def recover_from_rot(data, joints_num, skeleton):
    r_rot_quat, r_pos = recover_root_rot_pos(data)

    r_rot_cont6d = quaternion_to_cont6d(r_rot_quat)

    start_indx = 1 + 2 + 1 + (joints_num - 1) * 3
    end_indx = start_indx + (joints_num - 1) * 6
    cont6d_params = data[..., start_indx:end_indx]
    #     print(r_rot_cont6d.shape, cont6d_params.shape, r_pos.shape)
    cont6d_params = torch.cat([r_rot_cont6d, cont6d_params], dim=-1)
    cont6d_params = cont6d_params.view(-1, joints_num, 6)

    positions = skeleton.forward_kinematics_cont6d(cont6d_params, r_pos)

    return positions


def recover_from_ric(data, joints_num):
    r_rot_quat, r_pos = recover_root_rot_pos(data)
    positions = data[..., 4:(joints_num - 1) * 3 + 4]
    positions = positions.view(positions.shape[:-1] + (-1, 3))

    '''Add Y-axis rotation to local joints'''
    positions = qrot(qinv(r_rot_quat[..., None, :]).expand(positions.shape[:-1] + (4,)), positions)

    '''Add root XZ to joints'''
    positions[..., 0] += r_pos[..., 0:1]
    positions[..., 2] += r_pos[..., 2:3]

    '''Concate root and joints'''
    positions = torch.cat([r_pos.unsqueeze(-2), positions], dim=-2)

    return positions
# The given data is used to double check if you are on the right track.
reference1 = np.load('./HumanML3D/new_joints/012314.npy')
reference2 = np.load('./HumanML3D/new_joint_vecs/012314.npy')

'''
For HumanML3D Dataset
'''

if __name__ == "__main__":
    example_id = "000021"
    # Lower legs
    l_idx1, l_idx2 = 5, 8
    # Right/Left foot
    fid_r, fid_l = [8, 11], [7, 10]
    # Face direction, r_hip, l_hip, sdr_r, sdr_l
    face_joint_indx = [2, 1, 17, 16]
    # l_hip, r_hip
    r_hip, l_hip = 2, 1
    joints_num = 22
    # ds_num = 8
    data_dir = '/content/drive/MyDrive/Motion-dataset/joints'
    save_dir1 = './HumanML3D/new_joints/'
    save_dir2 = './HumanML3D/new_joint_vecs/'

    os.makedirs(save_dir1, exist_ok=True)
    os.makedirs(save_dir2, exist_ok=True)

    n_raw_offsets = torch.from_numpy(t2m_raw_offsets)
    kinematic_chain = t2m_kinematic_chain

    # Get offsets of target skeleton
    example_data = np.load(os.path.join(data_dir, example_id + '.npy'))
    example_data = example_data.reshape(len(example_data), -1, 3)
    example_data = torch.from_numpy(example_data)
    tgt_skel = Skeleton(n_raw_offsets, kinematic_chain, 'cpu')
    # (joints_num, 3)
    tgt_offsets = tgt_skel.get_offsets_joints(example_data[0])
    # print(tgt_offsets)

    source_list = os.listdir(data_dir)
    frame_num = 0
    for source_file in tqdm(source_list):
        source_data = np.load(os.path.join(data_dir, source_file))[:, :joints_num]
        try:
            data, ground_positions, positions, l_velocity = process_file(source_data, 0.002)
            rec_ric_data = recover_from_ric(torch.from_numpy(data).unsqueeze(0).float(), joints_num)
            np.save(pjoin(save_dir1, source_file), rec_ric_data.squeeze().numpy())
            np.save(pjoin(save_dir2, source_file), data)
            frame_num += data.shape[0]
        except Exception as e:
            print(source_file)
            print(e)
#         print(source_file)
#         break

    print('Total clips: %d, Frames: %d, Duration: %fm' %
          (len(source_list), frame_num, frame_num / 20 / 60))

  0%|          | 0/29232 [00:00<?, ?it/s]/content/HumanML3D/common/quaternion.py:398: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  v = torch.cross(v0, v1)
 16%|█▋        | 4769/29232 [19:44<36:24, 11.20it/s]

M009707.npy
cannot reshape array of size 0 into shape (0,newaxis)


 73%|███████▎  | 21436/29232 [38:41<13:33,  9.58it/s]

011059.npy
cannot reshape array of size 0 into shape (0,newaxis)


 84%|████████▎ | 24470/29232 [42:17<04:39, 17.03it/s]

M011059.npy
cannot reshape array of size 0 into shape (0,newaxis)


 85%|████████▌ | 24986/29232 [42:51<04:07, 17.16it/s]

009707.npy
cannot reshape array of size 0 into shape (0,newaxis)


100%|██████████| 29232/29232 [47:41<00:00, 10.21it/s]

Total clips: 29232, Frames: 4117392, Duration: 3431.160000m


In [ ]:
%cd HumanML3D

/content/HumanML3D


In [ ]:
import shutil

def move_directory(source_dir, target_dir):
    # Move the source directory to the target directory
    shutil.move(source_dir, target_dir)

source_directory = '/content/HumanML3D'
target_directory = '/content/drive/MyDrive/Motion-dataset'
move_directory(source_directory, target_directory)

In [ ]:
# @title cal_mean_variance
import numpy as np
import sys
import os
from os.path import join as pjoin


# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def mean_variance(data_dir, save_dir, joints_num):
    file_list = os.listdir(data_dir)
    data_list = []

    for file in file_list:
        data = np.load(pjoin(data_dir, file))
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / 1.0
    Std[1:3] = Std[1:3].mean() / 1.0
    Std[3:4] = Std[3:4].mean() / 1.0
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / 1.0

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]

    np.save(pjoin(save_dir, 'Mean.npy'), Mean)
    np.save(pjoin(save_dir, 'Std.npy'), Std)

    return Mean, Std
# The given data is used to double check if you are on the right track.
reference1 = np.load('/content/drive/MyDrive/Motion-dataset/HumanML3D/HumanML3D/Mean.npy')
reference2 = np.load('/content/drive/MyDrive/Motion-dataset/HumanML3D/HumanML3D/Std.npy')

if __name__ == '__main__':
    data_dir = '/content/drive/MyDrive/Motion-dataset/HumanML3D/HumanML3D/new_joint_vecs'
    save_dir = '/content/drive/MyDrive/Motion-dataset/HumanML3D/HumanML3D'
    mean, std = mean_variance(data_dir, save_dir, 22)
    print(mean)
    print(std)

#III Training

In [ ]:
!git clone https://github.com/Mael-zys/T2M-GPT.git

Cloning into 'T2M-GPT'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 156 (delta 35), reused 25 (delta 25), pack-reused 102
Receiving objects: 100% (156/156), 19.54 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
!bash /content/T2M-GPT/dataset/prepare/download_extractor.sh
!bash /content/T2M-GPT/dataset/prepare/download_glove.sh
!bash /content/T2M-GPT/dataset/prepare/download_model.sh
!bash /content/T2M-GPT/dataset/prepare/download_smpl.sh

Downloading...
From (original): https://drive.google.com/uc?id=1o7RTDQcToJjTm9_mNWTyzvZvjTWpZfug
From (redirected): https://drive.google.com/uc?id=1o7RTDQcToJjTm9_mNWTyzvZvjTWpZfug&confirm=t&uuid=e301abca-d08c-441a-908a-854995c239b1
To: /content/checkpoints/t2m.zip
100% 1.22G/1.22G [00:13<00:00, 89.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1KNU8CsMAnxFrwopKBBkC8jEULGLPBHQp
From (redirected): https://drive.google.com/uc?id=1KNU8CsMAnxFrwopKBBkC8jEULGLPBHQp&confirm=t&uuid=c0436844-b3eb-4146-8132-b804d6ca8eb3
To: /content/checkpoints/kit.zip
100% 705M/705M [00:13<00:00, 53.4MB/s]
Archive:  t2m.zip
   creating: t2m/
   creating: t2m/Comp_v6_KLD005/
   creating: t2m/Comp_v6_KLD005/meta/
  inflating: t2m/Comp_v6_KLD005/meta/mean.npy  
  inflating: t2m/Comp_v6_KLD005/meta/std.npy  
  inflating: t2m/Comp_v6_KLD005/opt.txt  
   creating: t2m/M2T_EL4_DL4_NH8_PS/
   creating: t2m/M2T_EL4_DL4_NH8_PS/meta/
  inflating: t2m/M2T_EL4_DL4_NH8_PS/meta/mean.npy  
  inflating: 

In [ ]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=dda1c62b1d58dbb9f0ad55370d588208088f8a8473932d40bacd6f198acee711
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [ ]:
!unzip /content/drive/MyDrive/Motion-dataset/HumanML3D/HumanML3D/texts.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: texts/M009616.txt       
  inflating: texts/M009617.txt       
  inflating: texts/M009618.txt       
  inflating: texts/M009619.txt       
  inflating: texts/M009620.txt       
  inflating: texts/M009621.txt       
  inflating: texts/M009622.txt       
  inflating: texts/M009623.txt       
  inflating: texts/M009624.txt       
  inflating: texts/M009625.txt       
  inflating: texts/M009626.txt       
  inflating: texts/M009627.txt       
  inflating: texts/M009628.txt       
  inflating: texts/M009629.txt       
  inflating: texts/M009630.txt       
  inflating: texts/M009631.txt       
  inflating: texts/M009632.txt       
  inflating: texts/M009633.txt       
  inflating: texts/M009634.txt       
  inflating: texts/M009635.txt       
  inflating: texts/M009636.txt       
  inflating: texts/M009637.txt       
  inflating: texts/M009638.txt       
  inflating: texts/M009639.txt       
  inflating: texts/M009640.txt 

In [ ]:
import shutil

def move_directory(source_dir, target_dir):
    # Move the source directory to the target directory
    shutil.move(source_dir, target_dir)

# Example usage
source_directory = '/content/T2M-GPT'
target_directory = '/content/drive/MyDrive/Motion-dataset'
move_directory(source_directory, target_directory)

In [ ]:
!python3 /content/T2M-GPT/train_vq.py \
--batch-size 256 \
--lr 2e-4 \
--total-iter 300000 \
--lr-scheduler 200000 \
--nb-code 512 \
--down-t 2 \
--depth 3 \
--dilation-growth-rate 3 \
--out-dir output \
--dataname t2m \
--vq-act relu \
--quantizer ema_reset \
--loss-vel 0.5 \
--recons-loss l1_smooth \
--exp-name VQVAE

2024-05-28 02:31:32.201108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 02:31:32.201165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 02:31:32.202512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-28 02:31:32.209511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 02:31:33.274410: W tensorflow/comp

KeyboardInterrupt: 